In [1]:
import os
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
from IPython.display import Image
data_type = 'float32'
os.environ['THEANO_FLAGS'] = 'device=cuda,floatX=' + data_type + ',force_device=True' + ',dnn.enabled=False'
import cell2location

## This model is required for WTA data
os.chdir('../code/')
import spacejam

Can not use cuDNN on context None: Disabled by dnn.enabled flag
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/software/team283/tl7/miniconda3/envs/spacejam/lib/python3.7/site-packages/theano/gpuarray/__init__.py", line 227, in <module>
    use(config.device)
  File "/software/team283/tl7/miniconda3/envs/spacejam/lib/python3.7/site-packages/theano/gpuarray/__init__.py", line 214, in use
    init_dev(device, preallocate=preallocate)
  File "/software/team283/tl7/miniconda3/envs/spacejam/lib/python3.7/site-packages/theano/gpuarray/__init__.py", line 159, in init_dev
    pygpu.blas.gemm(0, tmp, tmp, 0, tmp, overwrite_c=True)
  File "pygpu/blas.pyx", line 149, in pygpu.blas.gemm
  File "pygpu/blas.pyx", line 47, in pygpu.blas.pygpu_blas_rgemm
pygpu.gpuarray.GpuArrayException: (b'nvrtcCompileProgram: NVRTC_ERROR_INVALID_OPTION', 3)


In [2]:
from spacejam.models import LocationModelWTAMultiExperimentHierarchicalGeneLevel

models = [spacejam.models.LocationModelWTAMultiExperimentHierarchicalGeneLevel.LocationModelWTAMultiExperimentHierarchicalGeneLevel]

## Reload expression model

In [3]:
inf_aver=pd.read_csv('../data/C2L_RegressionModel_Mean_Expression_sc_ref_healthy.csv', sep=',', index_col=0)
inf_aver

,B.Cell,B.Plasma.IgA,B.Plasma.IgG,EC.Aerocyte,EC.Arterial,EC.Capillary,EC.Lymphatic,EC.Venous.Pul,EC.Venous.Sys,EP.AT1,...,Macro.IV,Monocyte,Mast,Mesothelial,NK,Pericytes,Proliferating.T.NK,T.CD4+,T.CD8+,Vascular.SMC
FO538757.2,0.000579,0.001639,0.001529,0.000060,0.000130,0.000032,0.000222,0.000156,0.001095,0.000081,...,0.000020,0.000027,0.000328,0.535061,0.000061,0.000112,0.000770,0.000102,0.000094,0.000184
AP006222.2,0.000554,0.001477,0.000967,0.000051,0.000161,0.000031,0.000208,0.000167,0.000963,0.000110,...,0.000021,0.000029,0.000282,0.721362,0.000058,0.000125,0.000625,0.000105,0.000074,0.000425
SAMD11,0.001515,0.002895,0.006690,0.000076,0.001764,0.000110,0.000358,0.016826,0.003472,0.000123,...,0.000089,0.000178,0.000310,0.709121,0.000073,0.000687,0.000588,0.000205,0.000104,0.008464
NOC2L,0.017657,0.082390,0.051832,0.002673,0.013651,0.000136,0.025665,0.006390,0.026931,0.015161,...,0.034019,0.013874,0.004577,0.998897,0.014968,0.004188,0.030006,0.015998,0.013244,0.039883
HES4,0.002111,0.001551,0.002401,0.006318,0.184139,0.024739,0.140539,0.003585,0.024953,0.115696,...,0.016290,0.064222,0.024419,0.671915,0.000707,0.392359,0.005227,0.003272,0.000118,0.476382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC002072.1,0.000729,0.288653,0.328073,0.000052,0.000135,0.000034,0.000233,0.000152,0.001058,0.000083,...,0.000017,0.000022,0.000254,0.959960,0.000051,0.000118,0.000599,0.000097,0.000077,0.000163
SEPTIN6,0.001274,0.001559,0.001321,0.000046,0.000110,0.000029,0.000181,0.000139,0.000886,0.000063,...,0.000018,0.000024,0.000231,0.827500,0.000046,0.000116,0.000632,0.000112,0.000071,0.000192
AL008633.1,0.000704,0.001307,0.001024,0.000058,0.000122,0.000031,0.000182,0.000150,0.000837,0.000071,...,0.000020,0.000020,0.000252,0.819836,0.000054,0.000120,0.000558,0.000108,0.000068,0.000171
IGF2.1,0.000668,0.001403,0.001206,0.000055,0.000309,0.000041,0.000347,0.000651,0.000839,0.000068,...,0.000019,0.000023,0.000204,0.978905,0.000048,0.000131,0.000588,0.000091,0.000071,0.000184


### Prepare Control data

In [4]:
adata = sc.read_h5ad('../data/AnnData_WTA_Healthy.h5ad' )

adata_neg = sc.read_h5ad('../data/AnnData_WTA_Healthy_NegProbes.h5ad')

nuclei_number=np.reshape(np.array(adata.obs['Nuclei'], dtype='float32'), (len(adata.obs['Nuclei']),1))

### Run Cell2Location

In [ ]:
for m in [models[0]]:
    cell2location.run_c2l.run_cell2location(inf_aver, adata, ignore_deprecation_exception=True,
            model_name=m,
            export_args={'path': '../data/',
                         'run_name_suffix': '_Healthy'},
            train_args={'use_raw': True, 'n_iter': 20000, 'sample_name_col': 'Donor_AltName'},
            model_kwargs={
            'Y_data' : adata_neg.X,
            'gene_level_prior': {'mean': 1 / 2, 'sd': 1 / 4, 'sample_alpha': 20},
            'cell_number_prior' : {'cells_per_spot': nuclei_number, 
                                   'factors_per_spot': 7, 'combs_per_spot': 7},
            'cell_number_var_prior' : {'cells_mean_var_ratio': 1e6, 
                                       'factors_mean_var_ratio': 100, 'combs_mean_var_ratio': 1}})